# Générer les paradigmes morphomiques 
- changer les noms de sortie pour avoir moins de timestamps


In [1]:
# -*- coding: utf8 -*-
import pandas as pd
import pickle, glob,re
import itertools as it
import networkx as nx
import datetime as dt

In [2]:
debug=False
noDiff=True
nbFormesPrint=False
regroupeTirages=True
phonologicalMap="-X"
cat="V"

In [3]:
pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
dierese={"j":"ij", "w":"uw","H":"yH","i":"ij","u":"uw","y":"yH"}
correctionsGlides={}
correctionsHiatus={}
preGlideFinal=set()

In [5]:
neutralisationsNORD=(u"6û",u"9ê")
neutralisationsSUD=(u"e2o",u"E9O")
if phonologicalMap=="-N":
    neutralisations=neutralisationsNORD
elif phonologicalMap=="-S":
    neutralisations=neutralisationsSUD
else:
    neutralisations=(u"",u"")
    phonologicalMap=("-X")
bdlexiqueIn = unicode(u"èò"+neutralisations[0])
bdlexiqueNum = [ord(char) for char in bdlexiqueIn]
neutreOut = unicode(u"EO"+neutralisations[1])
neutralise = dict(zip(bdlexiqueNum, neutreOut))

In [6]:
def recoder(chaine,table=neutralise):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    else:
        result=chaine
    return result

In [7]:
def checkFrench(prononciation):
    if prononciation:
        result=recoder(prononciation)
        m=re.match(ur"^.*([^ieèEaOouy926êôâ])[jwH]$",result)
        if m:
            print ("pb avec un glide final", prononciation)
        m=re.match(ur"(.*[ptkbdgfsSvzZ][rl])([jwH])(.*)",result)
        if m:
            n=re.search(ur"[ptkbdgfsSvzZ][rl](wa|Hi|wê)",result)
            if not n:
                glide=m.group(2)
                result=m.group(1)+dierese[glide]+m.group(3)
        m=re.match(ur"(.*)([iuy])([ieEaOouy].*)",result)
        if m:
            glide=m.group(2)
            result=m.group(1)+dierese[glide]+m.group(3)
        result=result.replace("Jj","J")
    else:
        result=prononciation
    return result

In [8]:
print checkFrench(u"f6ze")

f6ze


In [9]:
def lireLexique(nomLexique):
    with open(nomLexique, 'rb') as input:
        lexique=pickle.load(input)
    return lexique

In [10]:
def lexique2Paradigmes(lexique):
    return pd.pivot_table(lexique, values='phono', index=['lexeme'], columns=['case'], aggfunc=lambda x: ",".join(x)).reset_index().reindex()

In [11]:
coulMF=["orange","brown","yellow","lime",
          "green","teal","lightgray","pink","cyan","magenta"]
coulMT=["blue","darkgray","purple","red","olive","violet"]
couleurCase={
    "pi2S":"orange","pi3S":"orange",
    "ii1S":"brown","ii2S":"brown","ii3S":"brown","ii3P":"brown",
    "fi1S":"yellow","fi2P":"yellow",
    "pc1S":"yellow","pc2S":"yellow","pc3S":"yellow","pc3P":"yellow",
    "fi2S":"lime","fi3S":"lime",
    "fi1P":"green","fi3P":"green",
    "ps1S":"teal","ps2S":"teal","ps3S":"teal","ps3P":"teal",
    "ai2S":"lightgray","ai3S":"lightgray","is3S":"lightgray",
    "is1S":"pink","is2S":"pink","is3P":"pink",
    "ppMS":"cyan","ppMP":"cyan",
    "ppFS":"magenta","ppFP":"magenta"
            }
listLimites=[2,8,32,128,512]
listLimCoul=["red","pink","orange","green","teal","white"]

In [12]:
nomFichierMorphomes="/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/Longitudinales/PG-Longitudinal-Morphomes.tex"
listeTirages=glob.glob("/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/Longitudinales/Longitudinal-5[0-9]*.pkl")
listeVHF=[u"être","avoir","faire","dire","pouvoir","aller","voir","savoir","vouloir","venir"]

In [14]:
listeTirages

['/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/Longitudinales/Longitudinal-52-T800000-F36975.pkl',
 '/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/Longitudinales/Longitudinal-50-T600000-F33424.pkl',
 '/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/Longitudinales/Longitudinal-54-T1000000-F39898.pkl',
 '/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/Longitudinales/Longitudinal-53-T900000-F38562.pkl',
 '/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/Longitudinales/Longitudinal-55-T1100000-F41134.pkl',
 '/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/Longitudinales/Longitudinal-51-T700000-F35357.pkl']

In [15]:
def tiragesNom(nom):
    result=[tirage for tirage in listeTirages if nom in tirage]
    return result

In [16]:
def nbFormesEchantillons(nom):
    print nom
    for tirage in tiragesNom(nom):
        sample=lireLexique(tirage)
        print sample[sample["tir1"]>0]["tir1"].count(),
        print sample[sample["tir1"]>0]["tir1"].sum(),
        print tirage

In [17]:
def findFormNumbers(paradigme):
    dictNumbers={}
    cases=[c for c in paradigmes.columns.tolist() if c!="lexeme"]
    for case in cases:
        dictNumbers[case]=paradigme[case].count()
    return dictNumbers

In [18]:
def reduceTrad(el,traductions):
    if traductions[el]==el:
        return el
    else:
        return reduceTrad(traductions[el],traductions)
    
def defParadigme(paradigmes):
    syncretisms=[]
    cases=[c for c in paradigmes.columns.tolist() if c!="lexeme"]
    for (c1,c2) in it.combinations(cases, 2):
        c1Val=paradigmes[c1].notnull()
        c2Val=paradigmes[c2].notnull()
        c1Sur=paradigmes[c1].str.contains(",")
        c2Sur=paradigmes[c2].str.contains(",")
        l1=len(paradigmes[paradigmes[c1].notnull()])
        l2=len(paradigmes[paradigmes[c2].notnull()])
        paire=paradigmes[c1Val & c2Val & (paradigmes[c1]!=paradigmes[c2])][[c1,c2]]
        lenDiff=len(paire[~paire[c1].str.contains(",") & ~paire[c2].str.contains(",")])
        if lenDiff>0:
            if debug:
                print u"%s ≠ %s"%(c1,c2)
                print "différence",lenDiff
                if lenDiff<12:
                    print paire
        else:
            surAbondant=paire[paire[c1].str.contains(",") | paire[c2].str.contains(",")]
#            print "--------------------------------"
            if len(surAbondant)==0:
#                print u"%s = %s"%(c1,c2) 
                syncretisms.append(u"%s = %s"%(c1,c2))
            else:
                compatible=True
                for index,row in surAbondant.iterrows():
                    if "," in row[c1]:
                        if "," in row[c2]:
                            if row[c1]!=row[c2]:
                                compatible=False
                        else:
                            if not row[c2] in row[c1].split(","):
                                compatible=False
                    else:
                        if not row[c1] in row[c2].split(","):
                            compatible=False
                if compatible:
#                    print u"%s = %s"%(c1,c2)
                    syncretisms.append(u"%s = %s"%(c1,c2))
                else:
                    print u"%s ≠ %s"%(c1,c2)
                    print surAbondant

    reductionParadigme={c:c for c in cases}
    for syncretism in syncretisms:
        c1,c2=syncretism.split(" = ")
        removeC=max(c1,c2)
        keyC=min(c1,c2)
        if removeC in reductionParadigme:
            if keyC in reductionParadigme:
                key=min(reductionParadigme[keyC],reductionParadigme[removeC])
            else:
                key=min(keyC,reductionParadigme[removeC])
        else:
            if keyC in reductionParadigme:
                key=reductionParadigme[keyC]
            else:
                key=keyC
        reductionParadigme[removeC]=key
        reductionParadigme[keyC]=key

    for el in reductionParadigme:
        reductionParadigme[el]=reduceTrad(el,reductionParadigme)

    paradigmeReduit = {}
    for k, v in reductionParadigme.iteritems():
        paradigmeReduit[v] = paradigmeReduit.get(v, [])
        paradigmeReduit[v].append(k)
    print "syncrétismes",len(paradigmeReduit), sorted(paradigmeReduit.keys())
    print paradigmeReduit

In [19]:
def findClique(cliques,remElement="node"):
    pNodes=set()

    sCliques=sorted(cliques, key=len, reverse=True)

    cliqueFound=sorted(sCliques[0])
    pNodes=pNodes|set(sCliques[0])

    if remElement=="clique":
        removeCliques=[]
        for clique in sCliques:
            sClique=set(clique)
            if set.intersection(pNodes,sClique):
                removeCliques.append(clique)
        for clique in removeCliques:
            sCliques.remove(clique)
    elif remElement=="node":
        newCliques=[]
        for clique in sCliques:
            sClique=set(clique)
            diffClique=list(sClique-set.intersection(pNodes,sClique))
            if diffClique:
                newCliques.append(diffClique)               
#            print "sortie",newCliques
        sCliques=sorted(newCliques,key=len,reverse=True)
    else:
        print "remElement non prévu"
    return (cliqueFound,sCliques)

In [20]:
def findPartition(syncretismes):
    g=nx.Graph()
    for node in list(lexique[lexique["tir1"]>0]["case"].unique()):
        g.add_node(node)
    for syncretisme in syncretismes:
        c1,c2=syncretisme.split(" = ")
        g.add_edge(c1,c2)
    cliques.extend(list(nx.find_cliques(g)))
    sCliques=sorted(cliques, key=len, reverse=True)
    foundCliques=[]
    while sCliques:
        foundClique,sCliques=findClique(sCliques)
        foundCliques.append(foundClique)
    foundNodes=set(n for l in foundCliques for n in l )
    missingNodes=[[n] for n in g.nodes() if not n in foundNodes]
    partition=foundCliques+missingNodes
    dictPartition={l[0]:l for l in partition}
    return dictPartition

In [21]:
def drawPartition(dictPartition):
    g=nx.DiGraph()
    g.add_node("paradigme",color="red")
    for numMorphome,morphome in enumerate(dictPartition):
        g.add_node("M%02d"%numMorphome,color="blue")
        g.add_edge("paradigme","M%02d"%numMorphome)
        for case in dictPartition[morphome]:
            g.add_edge("M%02d"%numMorphome,case)
    graphs.append(g)

In [22]:
def findColours(dictPartition,debug=False):
    def assignColour(listMorphome,morphColour):
        for m in listMorphome:
            if not m in dictColours:
                dictColours[m]=morphColour
            else:
                print "conflit sur %s entre %s et %s"%(m,dictColours[m],morphColour)

    dictColours={}
    iCoul=0
    for morphome in dictPartition:
        listMorphome=dictPartition[morphome]
        lenMorphome=len(listMorphome)
        if lenMorphome>1:
            if debug: print lenMorphome,listMorphome,
            if morphome in couleurCase:
                if debug: print couleurCase[morphome]
                morphColour=couleurCase[morphome]
            else:
                noCoul=True
                for el in listMorphome:
                    if el in couleurCase:
                        if debug: print couleurCase[el]
                        morphColour=couleurCase[el]
                        noCoul=False
                        break
                if noCoul:
                    if debug: print "autre",coulMT[iCoul]
                    morphColour=coulMT[iCoul]
                    iCoul+=1
        else:
            morphColour="white"
        assignColour(listMorphome,morphColour)
    return dictColours

In [23]:
def makeTabular(dictColours,title="",coulLim=False, cat="V"):
    tabular=[]
    def makeLine6(tenseCode):
        line=[]
        for person in [per+nb for nb in ["S","P"] for per in ["1","2","3"]]:
            case=tenseCode+person
            if case in dictColours:
                line.append(r"\cellcolor{%s}%s"%(dictColours[case],case))
            else:
                line.append(r"\cellcolor{%s}%s"%("black",case))
        return r" & ".join(line)+r"\\"

    def makeLine3(tenseCode):
        line=[]
        for person in [per+nb for nb in ["S","P"] for per in ["1","2","3"]]:
            if person in ["2S","1P","2P"]:
                case=tenseCode+person
                if case in dictColours:
                    line.append(r"\cellcolor{%s}%s"%(dictColours[case],case))
                else:
                    line.append(r"\cellcolor{%s}%s"%("black",case))
#                line.append(r"\cellcolor{%s}%s"%(dictColours[case],case))
            else:
                line.append(r"---")
        return r" & ".join(line)+r"\\"
    
    def makeLineNF():
        line=[]
        for case in ["inf","pP","ppMS","ppMP","ppFS","ppFP"]:
            if case in dictColours:
                line.append(r"\cellcolor{%s}%s"%(dictColours[case],case))
            else:
                line.append(r"\cellcolor{%s}%s"%("black",case))
#            line.append(r"\cellcolor{%s}%s"%(dictColours[case],case))
        return r" & ".join(line)+r"\\"

    def makeLineMF(nombre):
        line=[]
        for genre in "mf":
            case=genre+nombre
            if case in dictColours:
                line.append(r"\cellcolor{%s}%s"%(dictColours[case],case))
            else:
                line.append(r"\cellcolor{%s}%s"%("black",case))
        return r" & ".join(line)+r"\\"
    
    def makeLineCoulLim():
        line=[]
        for numLimite,limite in enumerate(listLimites):
            line.append(r"\cellcolor{%s}%s"%(listLimCoul[numLimite],"$<$"+str(limite)))
        return r"\hline\hline "+r" & ".join(line)+r"\\"
        
    if cat=="V":
        top=[
            r"\begin{center}",
            r"\begin{tabular}{cccccc}",
            r"\hline"
            ]
        bottom=[
            r"\hline",
            r"\end{tabular}\\",
            title,
            r"\end{center}",
            r"\bigskip",
            r""
            ]
        tabular.append("\n".join(top))
        for tenseCode in ["pi","ii","fi","pc", "ps","ai", "is"]:
            tabular.append(makeLine6(tenseCode))
        tabular.append(makeLine3("pI"))
        tabular.append(makeLineNF())
    elif cat=="A":
        top=[
            r"\begin{center}",
            r"\begin{tabular}{cc}",
            r"\hline"
            ]
        bottom=[
            r"\hline",
            r"\end{tabular}\\",
            title,
            r"\end{center}",
            r"\bigskip",
            r""
            ]
        tabular.append("\n".join(top))
        for number in "sp":
            tabular.append(makeLineMF(number))
    if coulLim:
        tabular.append(makeLineCoulLim())
    tabular.append("\n".join(bottom))
    return "\n".join(tabular)

In [24]:
def caseEgalite(c1,c2,debug=False):
    c1Val=paradigmes[c1].notnull()
    c2Val=paradigmes[c2].notnull()
    c1Sur=paradigmes[c1].str.contains(",")
    c2Sur=paradigmes[c2].str.contains(",")
    l1=len(paradigmes[paradigmes[c1].notnull()])
    l2=len(paradigmes[paradigmes[c2].notnull()])
    egalPaire=paradigmes[c1Val & c2Val & (paradigmes[c1]==paradigmes[c2])][[c1,c2]]
    diffPaire=paradigmes[c1Val & c2Val & (paradigmes[c1]!=paradigmes[c2])][[c1,c2]]
    lenDiff=len(diffPaire[~diffPaire[c1].str.contains(",") & ~diffPaire[c2].str.contains(",")])
    if debug:
        print egalPaire
        print diffPaire
        print lenDiff
    if lenDiff>0:
        return False
        if debug:
            print u"%s ≠ %s"%(c1,c2)
            print "différence",lenDiff
            if lenDiff<12:
                print diffPaire
    else:
        surAbondant=diffPaire[diffPaire[c1].str.contains(",") | diffPaire[c2].str.contains(",")]
        if len(surAbondant)==0:
            if len(egalPaire)>0 or noDiff:
                if debug: print u"%s = %s"%(c1,c2), len(egalPaire)
                return True
        else:
            compatible=True
            egalCount=0
            for index,row in surAbondant.iterrows():
                if "," in row[c1]:
                    if "," in row[c2]:
                        setC1=set(row[c1].split(","))
                        setC2=set(row[c2].split(","))
                        print setC1&setC2
                        if not setC1&setC2:
                            compatible=False
                            break
                        else:
                            egalCount+=1
                    else:
                        if not row[c2] in row[c1].split(","):
                            compatible=False
                            break
                        else:
                            egalCount+=1
                else:
                    if not row[c1] in row[c2].split(","):
                        compatible=False
                        break
                    else:
                        egalCount+=1
            if compatible:
                if debug: print u"%s = %s"%(c1,c2), len(egalPaire), egalCount
                return True
            else:
                if debug: print u"%s ≠ %s"%(c1,c2)
                if debug: print surAbondant
                return False

In [25]:
def defParadigme(paradigmes,debug=False):
    syncretisms=[]

    cases=[c for c in paradigmes.columns.tolist() if c!="lexeme"]
    if debug: print "%d cases :"%len(cases),", ".join(cases)
    for (c1,c2) in it.combinations(cases, 2):
        if caseEgalite(c1,c2):
            syncretisms.append(u"%s = %s"%(c1,c2))
    dictPartition=findPartition(syncretisms)
    itemizeLines.append("%d morphomes"%len(dictPartition))
    for key in sorted(dictPartition):
        if debug: print "%s: %s," % (key, dictPartition[key]),
    if debug: print
    drawPartition(dictPartition)
    return(dictPartition)
    

In [26]:
def makeItemize(itemizeLines):
    lines=[]
    lines.append(r"\begin{itemize}")
    for line in itemizeLines:
        lines.append(r"\item "+line)
    lines.append(r"\end{itemize}")
    return "\n".join(lines)

In [27]:
def findNumberColour(dictNumbers):
    result={}
    for case in dictNumbers:
        if dictNumbers[case]<listLimites[0]:
            result[case]=listLimCoul[0]
        elif dictNumbers[case]<listLimites[1]:
            result[case]=listLimCoul[1]
        elif dictNumbers[case]<listLimites[2]:
            result[case]=listLimCoul[2]
        elif dictNumbers[case]<listLimites[3]:
            result[case]=listLimCoul[3]
        elif dictNumbers[case]<listLimites[4]:
            result[case]=listLimCoul[4]
        else:
            result[case]=listLimCoul[5]
    return result

In [28]:
def regroupeLexique(dictPartition,lexique):
    lexiqueRegroupe=lexique.copy()
    for p in dictPartition:
        lCases=dictPartition[p]
        lexiqueRegroupe.loc[lexiqueRegroupe["case"].isin(lCases),"morphome"]="/".join(lCases)
        lexiqueRegroupe.loc[lexiqueRegroupe["case"].isin(lCases),"case"]=p
    return lexiqueRegroupe.groupby(["lexeme","phono","case","morphome"]).agg({"freq":np.sum, "tir1":np.sum}).reset_index()

In [29]:
def tableauPaires(paradigme):
    table=pd.DataFrame(columns=["ligne","colonne","nbPaires"])
    refCases=paradigme.columns.tolist()
    refCases.remove("lexeme")
    for n,paire in enumerate(it.combinations_with_replacement(refCases,2)):
        if paire[0] in paradigme.columns and paire[1] in paradigme.columns:
            nbPaires=len(paradigme[[paire[0],paire[1]]].dropna())
        else:
            nbPaires=0
        table.loc[2*n]=[paire[0],paire[1],nbPaires]
        table.loc[2*n+1]=[paire[1],paire[0],nbPaires]
    tableau=table.pivot_table(index="ligne",columns=["colonne"])
    return tableau

In [30]:
graphs=[]
latexLines=[]
pairesTableaux=[]
#for nomLexique in sorted(listeTirages100k):
for nLexique,nomLexique in enumerate(listeTirages):
    lexique=lireLexique(nomLexique)
    lexique["phono"]=lexique["phono"].apply(checkFrench)
    if not 'tir1' in lexique.columns:
        lexique["tir1"]=1
    if not 'freq' in lexique.columns:
        lexique["freq"]=1
    lexique1=lexique[lexique["tir1"]>0]
    taille=lexique1["tir1"].count()
    itemizeLines=[]
    latexLines.append(nomLexique.split("/")[-1])
    itemizeLines.append("%d formes"%taille)
    itemizeLines.append("%d tirages"%lexique1["tir1"].sum())
    paradigmes=lexique2Paradigmes(lexique1)
    tableauOrigine=tableauPaires(paradigmes)
    dictNumbers=findFormNumbers(paradigmes)
    dictFormNumbers=findNumberColour(dictNumbers)

    syncretisms=[]
    cliques=[]

    dictPartition=defParadigme(paradigmes)
    if regroupeTirages and noDiff:
        lexiqueRegroupe=regroupeLexique(dictPartition,lexique1)
        paradigmesRegroupe=lexique2Paradigmes(lexiqueRegroupe)
        tableauRegroupe=tableauPaires(paradigmesRegroupe)

        tableauOrigine.to_csv(nomLexique.replace(".pkl","-%s%d-Separe.csv"%(dt.datetime.now().strftime("%y%m%d-%H%M"),nLexique)),sep=";",encoding="utf8")
        tableauRegroupe.to_csv(nomLexique.replace(".pkl","-%s%d-Groupe.csv"%(dt.datetime.now().strftime("%y%m%d-%H%M"),nLexique)),sep=";",encoding="utf8")

        pairesTableaux.append((tableauOrigine,tableauRegroupe))
        nomLexiqueRegroupe=nomLexique.replace(".pkl",phonologicalMap+"-%s-Morphomes.pkl"%dt.datetime.now().strftime("%y%m%d-%H%M"))
        if nomLexique==nomLexiqueRegroupe:
            print u"pb avec le nom du fichier pour les tirages regroupés"
        else:
            with open(nomLexiqueRegroupe,"wb") as output:
                pickle.dump(lexiqueRegroupe, output, pickle.HIGHEST_PROTOCOL)
    dictColours=findColours(dictPartition)
    latexLines.append(makeItemize(itemizeLines))
    latexLines.append(makeTabular(dictColours,title="Morphomes",cat=cat))
    if nbFormesPrint:
        latexLines.append(makeTabular(dictFormNumbers,title="Nombre de formes",coulLim=True,cat=cat))
if noDiff:
    nomFichierSortie=nomFichierMorphomes.replace(".tex",phonologicalMap+"-%s-NoDiff.tex"%dt.datetime.now().strftime("%y%m%d-%H%M"))
else:
    nomFichierSortie=nomFichierMorphomes.replace(".tex",phonologicalMap+"-%s-Diff.tex"%dt.datetime.now().strftime("%y%m%d-%H%M"))
with open(nomFichierSortie,"w") as output:
    output.write("\n".join(latexLines))

# TESTS

# Calculs pour le lexique complet

In [ ]:
lexiquePrefix="MGC-160104"
nomLexiqueBase="/Users/gilles/Box Sync/2015-Data/"+lexiquePrefix+'-Verbes2.pkl'
with open(nomLexiqueBase, 'rb') as input:
    lexiqueBase = pickle.load(input)

In [ ]:
lexiqueBase["freqcum"]= (lexiqueBase["freq"].cumsum()*1000).astype(int)
del lexiqueBase["ext"]
del lexiqueBase["cs"]
del lexiqueBase["ms"]
del lexiqueBase["vs"]
del lexiqueBase["prob"]
lexiqueBase["tir1"]=1

In [ ]:
graphs=[]
latexLines=[]
pairesTableaux=[]
lexique=lexiqueBase
lexique["phono"]=lexique["phono"].apply(checkFrench)
lexique1=lexique[lexique["tir1"]>0]
taille=lexique1["tir1"].count()
itemizeLines=[]
latexLines.append(nomLexiqueBase.split("/")[-1])
itemizeLines.append("%d formes"%taille)
itemizeLines.append("%d tirages"%lexique1["tir1"].sum())
paradigmes=lexique2Paradigmes(lexique1)
tableauBaseOrigine=tableauPaires(paradigmes)
dictNumbers=findFormNumbers(paradigmes)
dictFormNumbers=findNumberColour(dictNumbers)

syncretisms=[]
cliques=[]

dictPartition=defParadigme(paradigmes)
if regroupeTirages and noDiff:
    lexiqueBaseRegroupe=regroupeLexique(dictPartition,lexique1)
    paradigmesBaseRegroupe=lexique2Paradigmes(lexiqueBaseRegroupe)
    tableauBaseRegroupe=tableauPaires(paradigmesBaseRegroupe)

    tableauBaseOrigine.to_csv(nomLexiqueBase.replace(".pkl","-%s-Complet-Separe.csv"%(dt.datetime.now().strftime("%y%m%d-%H%M"))),sep=";",encoding="utf8")
    tableauBaseRegroupe.to_csv(nomLexiqueBase.replace(".pkl","-%s-Complet-Groupe.csv"%(dt.datetime.now().strftime("%y%m%d-%H%M"))),sep=";",encoding="utf8")

    pairesTableaux.append((tableauBaseOrigine,tableauBaseRegroupe))
    nomLexiqueBaseRegroupe=nomLexiqueBase.replace(".pkl",phonologicalMap+"-%s-Morphomes.pkl"%dt.datetime.now().strftime("%y%m%d-%H%M"))
    if nomLexiqueBase==nomLexiqueBaseRegroupe:
        print u"pb avec le nom du fichier pour les tirages regroupés"
    else:
        with open(nomLexiqueBaseRegroupe,"wb") as output:
            pickle.dump(lexiqueBaseRegroupe, output, pickle.HIGHEST_PROTOCOL)
dictColours=findColours(dictPartition)
latexLines.append(makeItemize(itemizeLines))
latexLines.append(makeTabular(dictColours,title="Morphomes",cat=cat))
if nbFormesPrint:
    latexLines.append(makeTabular(dictFormNumbers,title="Nombre de formes",coulLim=True,cat=cat))
if noDiff:
    nomFichierSortie=nomFichierMorphomes.replace(".tex",phonologicalMap+"-%s-NoDiff.tex"%dt.datetime.now().strftime("%y%m%d-%H%M"))
else:
    nomFichierSortie=nomFichierMorphomes.replace(".tex",phonologicalMap+"-%s-Diff.tex"%dt.datetime.now().strftime("%y%m%d-%H%M"))
with open(nomFichierSortie,"w") as output:
    output.write("\n".join(latexLines))

In [ ]:
caseEgalite("pi2S","pi3S",debug=True)

In [ ]:
for nEchant,(tableau1,tableau2) in enumerate(pairesTableaux):
    tableau1.to_csv(nomLexique.replace(".pkl","%d-Separe.csv"%nEchant),sep=";",encoding="utf8")
    tableau2.to_csv(nomLexique.replace(".pkl","%d-Groupe.csv"%nEchant),sep=";",encoding="utf8")

In [71]:
lexiqueRegroupe[["case","morphome"]]

,case,morphome
0,PRS,PRS
1,PLxPRSxINDx2,PLxPRSxINDx2
2,NFIN,NFIN/PLxPRSxINDx1/PLxPRSxINDx3/PLxPRSxOPTx1/PL...
3,PLxPSTxCONDxPFVx1,PLxPSTxCONDxPFVx1/PLxPSTxCONDxPFVx3/PLxPSTxSBJ...
4,PLxPSTxINDxPFVx1,PLxPSTxINDxPFVx1/PLxPSTxINDxPFVx3
5,SGxPSTxINDxPFVx2,SGxPSTxINDxPFVx2
6,NFIN,NFIN/PLxPRSxINDx1/PLxPRSxINDx3/PLxPRSxOPTx1/PL...
7,SGxPSTxINDxPFVx2,SGxPSTxINDxPFVx2
8,SGxPSTxINDxPFVx1,SGxPSTxINDxPFVx1/SGxPSTxINDxPFVx3
9,PLxPSTxCONDxPFVx2,PLxPSTxCONDxPFVx2/PLxPSTxSBJVxPFVx2
